In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import xmeans
from scipy.stats import skew, kurtosis

In [2]:
composition = pd.read_excel('data/raw/composicao_IBRx.xlsx')
# 2. Remove colunas "Company" e "Type"
composition = composition.drop(columns=["Company", "Type"])
# 3. Transforma de wide para long
date_cols = composition.columns.difference(['Code'], sort=False)
composition_long = composition.melt(
    id_vars='Code',
    value_vars=date_cols,
    var_name='Date',
    value_name='valores'
)
# 4. Converte as datas (formato "Dec-97") para datetime com mês e ano
composition_long['dates'] = pd.to_datetime(
    composition_long['Date'], format='%b-%y'
)
# 5. Remove a coluna original "Date" e reordena
composition_long = composition_long.drop(columns='Date')
composition_long = composition_long[['dates', 'Code', 'valores']]
# 6. Transforma de long para wide (tipo pivot_wider)
composition_wide = composition_long.pivot(
    index='dates',
    columns='Code',
    values='valores'
).reset_index()
composition = composition_wide.copy()

In [3]:
stocks  = pd.read_excel('data/raw/economatica_b3.xlsx')
# 2. Substituir os nomes dos meses em português por números
meses_pt = {
    "Jan": "01", "Fev": "02", "Mar": "03", "Abr": "04", "Mai": "05", "Jun": "06",
    "Jul": "07", "Ago": "08", "Set": "09", "Out": "10", "Nov": "11", "Dez": "12"
}
for pt, num in meses_pt.items():
    stocks["Data"] = stocks["Data"].str.replace(pt, num, regex=False)

# 3. Converter para datetime (assumindo formato "MM-YYYY" após substituições)
stocks["dates"] = pd.to_datetime(stocks["Data"], format="%m-%Y")

# 4. Reorganizar colunas e remover a original
stocks = stocks.drop(columns="Data")
stocks = stocks[["dates"] + [col for col in stocks.columns if col != "dates"]]

# 5. Filtrar intervalo de datas
stocks = stocks[(stocks["dates"] >= "1997-12-01") & (stocks["dates"] <= "2023-12-01")]

# 6. Renomear colunas removendo prefixo repetitivo
stocks.columns = stocks.columns.str.replace(
    r"Retorno\ndo fechamento\nem 1 meses\nEm moeda orig\najust p/ prov\n", "", regex=True
)
stocks.replace("-", np.nan, inplace=True)

C:\Users\richard.souza\AppData\Local\Temp\ipykernel_9640\120970321.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  stocks.replace("-", np.nan, inplace=True)


In [ ]:
InS = 120                           # tamanho janela 
OoS = stocks.shape[0] - InS         # janera de teste
p = stocks.shape[1] - 1             # numero de colunas
nmethods = 10                       # quantidade de simulações da matriz de cov (nao usado)

In [ ]:
tickers = [i for i in range(p)]
w = []
Rport = pd.DataFrame(index=range(OoS), columns=['x_means'])         # dataframe de retornos
to = pd.DataFrame(index=range(OoS - 1), columns=['x_means'])        # dataframe de turnover
sspw = pd.DataFrame(index=range(OoS), columns=['x_means'])          # dataframe de concentracao de pesos
w_xmeans_full = pd.DataFrame(index=range(OoS), columns=tickers)     # dataframe de pesos para seus tickers

In [ ]:
def get_composition(composition: pd.DataFrame, i: int, InS: int, date_ins: pd.Series) -> pd.Series:
    # 1. Recorte das linhas
    df_slice = composition.iloc[i:(InS - 1 + i)]
    # 2. Filtra por data até a data final de interesse
    df_filtered = df_slice[df_slice["dates"] <= date_ins.iloc[-1]]
    # 3. Última linha do filtro
    last_row = df_filtered.tail(1)
    # 4. Remove a coluna "dates"
    last_row_no_dates = last_row.drop(columns="dates")
    # 5. Conta valores NaN por coluna
    return last_row_no_dates.isna().sum()

def calculate_to(previous_weights, desired_weights, oos_returns, p):
    # Substitui NaNs por 0 nos retornos
    oos_returns_ = oos_returns.fillna(0)

    # Atualiza os pesos com base nos retornos
    num = previous_weights * (1 + oos_returns_ / 100)
    den = np.nansum(num)  # soma ignorando NaNs

    updated_weights = num / den #if den != 0 else np.zeros_like(num)

    # Calcula o turnover como soma das diferenças absolutas
    t_o = np.sum(np.abs(desired_weights - updated_weights), axis=1)

    return t_o.tolist()[0]

for i in range(OoS):
    # obtendo as datas que serao usadas nessa janela
    date_ins = stocks.iloc[i:(InS - 1 + i), ]['dates']
    # indo na tabela composition que é a tabela das açoes mais liquidas do brasil
    # e pegando os ativos que estão dentro do intervalo de data e dentro da janela
    # a funcao get_composition retorna a soma de valores nan nessa janela
    aux1 = get_composition(composition, i, InS, date_ins)
    # indo na tabela de ações e pegando ativos dessa janela e removendo a coluna dates
    # para ter somente as colunas de retornos e calcula quantos valores nan tem essas
    # colunas de retorno 
    aux2 = stocks.iloc[i:(InS - 1 + i)].drop(columns="dates").isna().sum()
    # aqui pega as colunas onde não existem nan em aux1 e aux2, depois pega a intersecao
    aux = list(set(aux1[aux1 == 0].index).intersection(aux2[aux2 == 0].index))
    # pega os tickers que estao presente em aux que nao apresentam valores nan e então
    # vai na tabela de retornos e pega os retornos desses ativos, nessa janela
    retu_ins = stocks.iloc[i:(InS - 1 + i)][aux]#.to_numpy()
    # pega os retornos no mes seguinte dos ativos presentes em aux, esse retorno do mes
    # seguinte é o retorno fora da amostra que usaremos para calcular a performance do
    # portfolio
    r_oos = stocks.iloc[InS + i, ][aux].to_numpy()
    # guarda os retornos fora da amostra em um formato dataframe
    r_oos_full = pd.DataFrame(np.nan, index=[0], columns=stocks.columns[1:])
    r_oos_full.loc[0, aux] = r_oos
    # definindo uma seed diferente a cada iteração
    seed = np.random.seed(i)
    # lista contendo os ativos 
    asset = retu_ins.columns.tolist()
    print(f'começando xmeans: {i}')
    # aqui performar o xmeans e pegar os pesos dos ativos
    w_xmeans = xmeans.main(retu_ins, np.cov(retu_ins, rowvar=False), asset, seed)
    # lista para guardar em formato de dataframe os pesos que o xmeans retorna 
    w.append(pd.DataFrame([w_xmeans]))
    # armazenando o retorno fora da amostra para o portfolio
    Rport.loc[i, :] = [(w_xmeans @ r_oos.T)]
    # armazenando os pesos elevado ao quadrado para medir a concentração dos pesos no portfólio
    sspw.loc[i, :]  = [np.sum(w_xmeans**2)]
    # armazenar no dataframe os pesos que o xmeans retornou, porém dessa vez 
    # vinculando o peso aos tickers
    w_xmeans_full.loc[i, aux] = w_xmeans

    if i > 2:
        # calculos de turnouver
        to.loc[i - 1, :] = calculate_to(
            w_xmeans_full.loc[i - 1, :],
            w_xmeans_full.loc[i, :],
            r_oos_full,
            p
        )

começando xmeans: 0
começando xmeans: 1
começando xmeans: 2
começando xmeans: 3
começando xmeans: 4
começando xmeans: 5
começando xmeans: 6
começando xmeans: 7
começando xmeans: 8
começando xmeans: 9
começando xmeans: 10
começando xmeans: 11
começando xmeans: 12
começando xmeans: 13
começando xmeans: 14
começando xmeans: 15
começando xmeans: 16
começando xmeans: 17
começando xmeans: 18
começando xmeans: 19
começando xmeans: 20
começando xmeans: 21
começando xmeans: 22
começando xmeans: 23
começando xmeans: 24
começando xmeans: 25
começando xmeans: 26
começando xmeans: 27
começando xmeans: 28
começando xmeans: 29
começando xmeans: 30
começando xmeans: 31
começando xmeans: 32
começando xmeans: 33
começando xmeans: 34
começando xmeans: 35
começando xmeans: 36
começando xmeans: 37
começando xmeans: 38
começando xmeans: 39
começando xmeans: 40
começando xmeans: 41
começando xmeans: 42
começando xmeans: 43
começando xmeans: 44
começando xmeans: 45
começando xmeans: 46
começando xmeans: 47
co

In [ ]:
def medidas(x, rf=0):
    x = pd.Series(x).astype(float)
    # Retorno médio e desvio padrão
    AV = x.mean()
    SD = x.std()
    # Sharpe Ratio
    SR = (AV - rf) / SD if SD != 0 else np.nan
    # Adjusted Sharpe Ratio
    sk = skew(x.dropna(), bias=False)
    kt = kurtosis(x.dropna(), fisher=True, bias=False)
    ASR = SR * (1 + (sk / 6) * SR - (kt / 24) * SR**2) if not np.isnan(SR) else np.nan
    # Sortino Ratio
    downside_diff = x - rf
    downside_risk = np.sqrt(np.mean(np.where(downside_diff < 0, downside_diff**2, 0)))
    SO = (AV - rf) / downside_risk if downside_risk != 0 else np.nan
    # Anualização
    output = [12 * AV, np.sqrt(12) * SD, np.sqrt(12) * SR, np.sqrt(12) * ASR, np.sqrt(12) * SO]
    return pd.Series(output, index=["AV", "SD", "SR", "ASR", "SO"])

linha_medidas = medidas(Rport['x_means'])
linha_to = pd.Series([to['x_means'].mean()], index=['TO'])
linha_sspw = pd.Series([sspw['x_means'].mean()], index=['SSPW'])

oos_results = pd.DataFrame(pd.concat([linha_medidas, linha_to, linha_sspw]), columns=["x_means"]).T
oos_results

,AV,SD,SR,ASR,SO,TO,SSPW
x_means,9.392192,19.893116,0.472133,0.47145,0.802692,0.653605,0.038024


In [53]:
Rport

,x_means
0,-1.616359
1,-9.506464
2,NaN
3,-4.177332
4,8.662982
...,...
188,-5.387625
189,-0.847994
190,-4.554174
191,14.637164


In [54]:
to

,x_means
0,NaN
1,NaN
2,0.658466
3,0.473583
4,0.442323
...,...
187,0.793002
188,0.88652
189,0.848372
190,0.551281


In [55]:
sspw

,x_means
0,0.035077
1,0.047984
2,0.049282
3,0.037894
4,0.047414
...,...
188,0.083972
189,0.034108
190,0.023383
191,0.051076
